In [113]:
import astropy.io.ascii as ascii
import numpy as np
import pyvo
import astroquery
from astroquery.irsa import Irsa
import aplpy
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u
from astropy.units import Quantity


In [22]:
Irsa.list_catalogs()   

{'allwise_p3as_psd': 'AllWISE Source Catalog',
 'allwise_p3as_mep': 'AllWISE Multiepoch Photometry Table',
 'allwise_p3as_psr': 'AllWISE Reject Table',
 'allwise_p3as_cdd': 'AllWISE Atlas Metadata Table',
 'allwise_p3am_xrf': 'AllWISE Frame Cross-Reference Table',
 'allwise_p3al_lod': 'AllWISE Atlas Inventory Table',
 'allwise_p3am_cdd': 'AllWISE Atlas Image Inventory Table',
 'allwise_mfpos': 'AllWISE Refined Pointing Information for the Single-exposure Images',
 'neowiser_p1bs_psd': 'NEOWISE-R Single Exposure (L1b) Source Table',
 'neowiser_p1ba_mch': 'NEOWISE-R Known Solar System Object Possible Association List ( Caution )',
 'neowiser_p1bs_frm': 'NEOWISE-R Single Exposure (L1b) Frame Metadata Table',
 'neowiser_p1bl_lod': 'NEOWISE-R Single Exposure (L1b) Scan Inventory Table',
 'neowiser_p1bm_frm': 'NEOWISE-R Single Exposure (L1b) Image Inventory Table',
 'allsky_4band_p3as_psd': 'WISE All-Sky Source Catalog',
 'allsky_4band_p1bs_psd': 'WISE All-Sky Single Exposure (L1b) Source Ta

In [54]:
Irsa.ROW_LIMIT = 1000 

In [70]:
 table = Irsa.query_region(catalog='fp_psc', spatial='All-Sky')

In [81]:
help(Irsa)

Help on IrsaClass in module astroquery.irsa.core object:

class IrsaClass(astroquery.query.BaseQuery)
 |  This is the base class for all the query classes in astroquery. It
 |  is implemented as an abstract class and must not be directly instantiated.
 |  
 |  Method resolution order:
 |      IrsaClass
 |      astroquery.query.BaseQuery
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  list_catalogs(self)
 |      Return a dictionary of the catalogs in the IRSA Gator tool.
 |      
 |      Returns
 |      -------
 |      catalogs : dict
 |          A dictionary of catalogs where the key indicates the catalog
 |          name to be used in query functions, and the value is the verbose
 |          description of the catalog.
 |  
 |  print_catalogs(self)
 |      Display a table of the catalogs in the IRSA Gator tool.
 |  
 |  query_region(self, coordinates=None, catalog=None, spatial='Cone', radius=<Quantity 10. arcsec>, width=None, polygon=None, get_query_payload=False, ver

In [73]:
table = Irsa.query_region(catalog='allwise_p3am_cdd ', spatial = 'All-Sky')

In [75]:
table = Irsa.query_region(catalog='neowiser_p1bm_frm', spatial = 'All-Sky')

In [97]:
# table.info

In [35]:
# build coordinate:
coord1 = SkyCoord(ra='04 39 53.97816' , dec = '+26 03 09.6804', unit=(u.hour, u.deg))

In [37]:
coord1

<SkyCoord (ICRS): (ra, dec) in deg
    (69.974909, 26.052689)>

In [40]:
coord1.ra.deg

69.974909

AttributeError: 'IrsaClass' object has no attribute 'list_votable_fields'

In [92]:
 table = Irsa.query_region(coord1,
                           catalog='fp_psc', spatial='Box',
                           width=5 * u.arcmin)

In [95]:
 table = Irsa.query_region(coord1,
                           catalog='allwise_p3am_cdd', spatial='Box',
                           width=100 * u.arcmin)

TableParseError: Failed to parse IRSA votable! The raw response can be found in self.response, and the error in self.table_parse_error.

In [50]:
table

ra,dec,clon,clat,err_maj,err_min,err_ang,designation,j_m,j_cmsig,j_msigcom,j_snr,h_m,h_cmsig,h_msigcom,h_snr,k_m,k_cmsig,k_msigcom,k_snr,ph_qual,rd_flg,bl_flg,cc_flg,ndet,gal_contam,mp_flg,hemis,xdate,scan,glon,glat,a,dist_opt,phi_opt,b_m_opt,vr_m_opt,nopt_mchs,ext_key,j_h,h_k,j_k
deg,deg,,,arcsec,arcsec,deg,,mag,mag,mag,,mag,mag,mag,,mag,mag,mag,,,,,,,,,,,,deg,deg,,arcsec,deg,mag,mag,,,,,
float64,float64,object,object,float64,float64,int32,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,object,object,object,object,int32,int32,object,object,int32,float64,float64,object,float64,int32,float64,float64,int32,int32,float64,float64,float64
69.935602,26.078903,04h39m44.54s,26d04m44.05s,0.07,0.06,90,04394454+2604440,15.173,0.044,0.046,31.9,13.735,0.029,0.030,53.1,13.026,0.034,0.035,48.5,AAA,222,111,000,663366,0,0,n,1998-10-14,91,173.780,-13.534,0,--,--,--,--,0,--,1.438,0.709,2.147
69.996561,26.088877,04h39m59.17s,26d05m19.96s,0.17,0.16,135,04395917+2605199,17.127,--,--,--,16.251,0.217,0.217,5.2,14.975,0.133,0.133,8.1,UCB,022,011,000,000606,0,0,n,1998-10-14,92,173.808,-13.486,0,--,--,--,--,0,--,--,1.276,--
70.001990,26.086060,04h40m00.48s,26d05m09.82s,0.11,0.11,135,04400047+2605098,17.940,--,--,--,15.556,0.119,0.120,9.9,13.969,0.052,0.053,20.4,UBA,022,011,000,000646,0,0,n,1998-10-14,92,173.813,-13.484,0,--,--,--,--,0,--,--,1.587,--
69.979855,26.089634,04h39m55.17s,26d05m22.68s,0.07,0.06,83,04395516+2605226,15.860,0.068,0.069,16.9,13.347,0.029,0.030,75.9,12.247,0.023,0.024,99.5,AAA,222,111,000,166666,0,0,n,1998-10-14,91,173.797,-13.497,0,--,--,--,--,0,--,2.513,1.1,3.613
69.987841,26.090353,04h39m57.08s,26d05m25.27s,0.09,0.08,90,04395708+2605252,16.288,0.100,0.101,11.4,14.149,0.043,0.043,36.2,13.205,0.038,0.039,41.1,AAA,222,111,000,066655,0,0,n,1998-10-14,91,173.801,-13.491,0,--,--,--,--,0,--,2.139,0.944,3.083
69.937014,26.031330,04h39m44.88s,26d01m52.79s,0.07,0.06,90,04394488+2601527,10.640,0.018,0.022,2072.2,9.515,0.019,0.021,2587.7,8.950,0.016,0.018,2072.1,AAA,222,111,000,664466,0,0,n,1998-10-14,91,173.818,-13.564,U,1.2,158,19.30,16.20,1,--,1.125,0.565,1.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69.954665,26.025644,04h39m49.12s,26d01m32.32s,0.24,0.21,103,04394911+2601323,17.086,--,--,--,15.913,0.165,0.165,7.1,15.298,0.173,0.173,6.0,UCC,022,011,000,000606,0,0,n,1998-10-14,91,173.833,-13.555,0,--,--,--,--,0,--,--,0.615,--


In [98]:
coord1

<SkyCoord (ICRS): (ra, dec) in deg
    (69.974909, 26.052689)>

In [105]:

service = pyvo.dal.TAPService('https://irsa.ipac.caltech.edu/TAP')
result = service.run_async("""
           SELECT *
           FROM allwise_p3am_cdd
           WHERE CONTAINS(POINT('ICRS',ra, dec), CIRCLE('ICRS',202.48417,47.23056,0.4))=1
    """)
tab = result.to_table()

DALQueryError: Query Error

In [110]:

service = pyvo.dal.TAPService('https://irsa.ipac.caltech.edu/TAP')
result = service.run_async("""
           SELECT TOP 10
           FROM allwise_p3am_cdd
    """)
tab = result.to_table()

DALServiceError: Cannot wait for job completion. Job is not active!

In [114]:
# build coordinate:
coord1 = SkyCoord(ra='04 39 53.97816' , dec = '+26 03 09.6804', unit=(u.hour, u.deg))
size =  Quantity(5, unit = u.arcsec)

In [116]:
tap_service = pyvo.dal.TAPService('https://irsa.ipac.caltech.edu/TAP')

tap_results = tap_service.search("SELECT TOP 10 * FROM Allwise_p3am_cdd")

In [119]:
tap_results

<Table length=10>
   coadd_id    band naxis ...          y                   z           spt_ind 
                          ...                                                  
    object    int64 int64 ...       float64             float64         int64  
------------- ----- ----- ... ------------------- ------------------- ---------
0686p439_ac51     4     2 ...  0.6709811724712000  0.6936393384524679 233233013
0690p605_ac51     1     2 ...  0.4586171138312100  0.8710233500179300 231230333
0690p605_ac51     2     2 ...  0.4586171138312100  0.8710233500179300 231230333
0690p605_ac51     3     2 ...  0.4586171138312100  0.8710233500179300 231230333
0690p605_ac51     4     2 ...  0.4586171138312100  0.8710233500179300 231230333
0691p590_ac51     1     2 ...  0.4803297386018280  0.8577360842400100 231230111
0691p590_ac51     2     2 ...  0.4803297386018280  0.8577360842400100 231230111
0691p590_ac51     3     2 ...  0.4803297386018280  0.8577360842400100 231230111
0691p590_ac51     4   

In [118]:
print(tap_service.maxrec)


2000000000
